# Fine-Tuning Model dan Pengujian dengan Gradio di Cloud

Notebook ini akan:
1. Menginstal dependensi yang diperlukan.
2. Membuat file `.jsonl` dari file PDF (jika ada).
3. Melakukan fine-tuning model bahasa menggunakan LoRA.
4. Menjalankan antarmuka Gradio untuk menguji model.

## Langkah 1: Instalasi Dependensi

Jalankan sel ini untuk menginstal semua dependensi yang diperlukan.

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install transformers datasets peft gradio pdfplumber ollama

## Langkah 2: Unggah File PDF (Opsional)

Jika Anda memiliki file PDF untuk membuat dataset `.jsonl`, unggah file PDF Anda di sini. Jika tidak, lewati langkah ini dan pastikan Anda memiliki file `custom_knowledge.jsonl` yang sudah ada.

Catatan: Anda juga dapat mengunggah file `custom_knowledge.jsonl` secara manual jika tidak ingin membuatnya dari PDF.

In [ ]:
from google.colab import files
import os

# Buat direktori untuk file PDF
pdf_dir = "./pdfs"
os.makedirs(pdf_dir, exist_ok=True)

# Unggah file PDF atau custom_knowledge.jsonl
uploaded = files.upload()

# Pindahkan file yang diunggah ke direktori ./pdfs (jika PDF)
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        os.rename(filename, os.path.join(pdf_dir, filename))
        print(f"File {filename} berhasil diunggah ke {pdf_dir}")
    elif filename == 'custom_knowledge.jsonl':
        print(f"File {filename} berhasil diunggah.")
    else:
        print(f"File {filename} diabaikan (harus .pdf atau custom_knowledge.jsonl)")

## Langkah 3: Membuat File `.jsonl` dari PDF (Opsional)

Jalankan sel ini untuk membuat file `custom_knowledge.jsonl` dari file PDF yang diunggah. Jika tidak ada file PDF, langkah ini akan dilewati. Pastikan Ollama sudah terinstal dan model (misalnya `llama3.1`) sudah diunduh.

**Catatan**: Langkah ini membutuhkan Ollama untuk menghasilkan pasangan pertanyaan dan jawaban. Jika Anda menjalankan di Colab, Anda perlu menginstal Ollama secara manual (misalnya, melalui `!ollama pull llama3.1`). Namun, karena Colab tidak mendukung Ollama secara langsung, Anda mungkin perlu menjalankan langkah ini di lokal terlebih dahulu untuk membuat file `.jsonl`, lalu mengunggahnya di langkah sebelumnya.

In [ ]:
import os
import json
import pdfplumber
import ollama

# Konfigurasi
pdf_directory = "./pdfs"
output_jsonl = "custom_knowledge.jsonl"
min_data = 5000
ollama_model = "llama3.1"

# Fungsi untuk membaca teks dari file PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Fungsi untuk menghasilkan pertanyaan dan jawaban menggunakan Ollama
def generate_qa(text):
    prompt = f"Generate a question and answer pair based on the following text in English:\n\n{text}\n\nFormat the output as JSON with 'prompt' and 'completion' keys."
    response = ollama.chat(model=ollama_model, messages=[{'role': 'user', 'content': prompt}])
    try:
        qa = json.loads(response['message']['content'])
        return qa
    except json.JSONDecodeError:
        return None

# Fungsi untuk membuat file .jsonl
def create_jsonl(data, output_file):
    with open(output_file, 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

# Fungsi utama untuk membuat .jsonl
def generate_jsonl():
    print(f"Memeriksa file PDF di direktori {pdf_directory}...")
    pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith('.pdf')]
    if not pdf_files:
        print("Tidak ada file PDF. Melewati langkah pembuatan .jsonl.")
        return

    all_qa = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_directory, pdf_file)
        text = extract_text_from_pdf(pdf_path)
        text_chunks = [text[i:i+500] for i in range(0, len(text), 500)]
        for chunk in text_chunks:
            qa = generate_qa(chunk)
            if qa and 'prompt' in qa and 'completion' in qa:
                all_qa.append(qa)

    # Ulangi data jika kurang dari 5000
    while len(all_qa) < min_data:
        all_qa.extend(all_qa[:min_data - len(all_qa)])

    create_jsonl(all_qa, output_jsonl)
    print(f"File {output_jsonl} berhasil dibuat dengan {len(all_qa)} entri.")

# Jalankan fungsi
generate_jsonl()

## Langkah 4: Fine-Tuning Model

Jalankan sel ini untuk melakukan fine-tuning model. Pastikan file `custom_knowledge.jsonl` sudah ada (dibuat di langkah sebelumnya atau diunggah secara manual).

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType

# Paksa penggunaan CPU-only
device = torch.device("cpu")

# Load model dan tokenizer dengan attn_implementation="eager"
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager"
)
model.to(device)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Konfigurasi LoRA untuk efisiensi
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, lora_config)

# Load dataset
dataset = load_dataset('json', data_files="custom_knowledge.jsonl", split='train')

# Preprocessing dataset
def preprocess_function(examples):
    texts = [f"{p.strip()} [SEP] {c.strip()}" for p, c in zip(examples["prompt"], examples["completion"])]
    tokenized = tokenizer(texts, truncation=True, max_length=256, padding="max_length", return_tensors="pt")
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments yang dioptimalkan untuk CPU
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    save_steps=100,
    save_total_limit=2,
    logging_steps=20,
    learning_rate=5e-5,
    warmup_steps=50,
    fp16=False,
    optim="adamw_torch",
    eval_strategy="no",
    report_to="none",
)

# Inisialisasi trainer dengan label_names
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    label_names=["labels"],
)

# Mulai fine-tuning
trainer.train()

# Simpan model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
# Gabungkan adapter LoRA dan simpan versi merged
model = model.merge_and_unload()
model.save_pretrained("./fine_tuned_model_merged")
tokenizer.save_pretrained("./fine_tuned_model_merged")

## Langkah 5: Jalankan Antarmuka Gradio untuk Pengujian

Jalankan sel ini untuk memuat model yang telah di-fine-tune dan menguji model melalui antarmuka Gradio.

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessor, LogitsProcessorList

# Device setup
device = torch.device("cpu")

# Load model and tokenizer
model_path = "./fine_tuned_model_merged"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to(device)
model.eval()

# Custom logits processor for stability
class SafeLogitsProcessor(LogitsProcessor):
    def __call__(self, input_ids, scores):
        scores = torch.where(
            torch.isnan(scores) | torch.isinf(scores),
            torch.full_like(scores, -1e9),
            scores
        )
        return scores

# Generate response function
def generate_response(prompt, max_new_tokens=100, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    input_length = inputs["input_ids"].shape[1]
    logits_processor = LogitsProcessorList([SafeLogitsProcessor()])
    
    try:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                logits_processor=logits_processor,
            )
        generated_tokens = outputs[0][input_length:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        return response.strip()
    except Exception as e:
        print(f"Error: {str(e)}")
        return "Sorry, an error occurred."

# Chat interface
def chat_interface(user_input, history):
    if history:
        prompt = "\n".join([f"User: {h[0]}\nBot: {h[1]}" for h in history]) + f"\nUser: {user_input}\nBot:"
    else:
        prompt = f"User: {user_input}\nBot:"
    return generate_response(prompt)

# Gradio UI
with gr.Blocks(title="Chatbot Fine-Tuned Model") as demo:
    gr.Markdown("# Chatbot Fine-Tuned Model\nAsk anything!")
    chatbot = gr.Chatbot(label="Conversation")
    user_input = gr.Textbox(label="Your Question", placeholder="Type here...")
    submit_btn = gr.Button("Send")
    state = gr.State([])

    def submit_message(user_input, history):
        if not user_input.strip():
            return "", history
        bot_response = chat_interface(user_input, history)
        history.append((user_input, bot_response))
        return "", history

    submit_btn.click(fn=submit_message, inputs=[user_input, state], outputs=[user_input, chatbot])
    user_input.submit(fn=submit_message, inputs=[user_input, state], outputs=[user_input, chatbot])

demo.launch(share=True)